# POTATO Take-Home Task

# Overview
This notebook processes Twitter data from TSV files, allowing for various queries based on user input.

## Part 1: Data Ingestion



# Import Libraries

In [2]:
pip install pandas elasticsearch flask


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/524.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/524.6 kB ? eta -:--:--
   ------------------- -------------------- 262.1/524.6 kB ? eta -:--:--
   ------------------- -------------------- 262.1/524.6 kB ? eta -:--:--
   ------------------- -------------------- 262.1/524.6 kB ? eta -:--:--
   -------------------------------------- 524.6/524.6 kB 494.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from elasticsearch import Elasticsearch

In [11]:
# Load the data
tweets_df = pd.read_csv('correct_twitter_201904.tsv', sep='\t')


In [6]:
tweets_df.head()

,id,event,ts1,ts2,from_stream,directly_from_stream,from_search,directly_from_search,from_quote_search,directly_from_quote_search,...,retweeted,retweeted_author_id,retweeted_handle,retweeted_follower_count,mentioned_author_ids,mentioned_handles,hashtags,urls,media_keys,place_id
0,1131594960443199488,britney_201904,2022-02-28 09:34:44.627023-05:00,2022-02-28 09:34:44.627023-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1131594976750653440,britney_201904,2022-02-28 09:34:44.626921-05:00,2022-02-28 09:34:44.626921-05:00,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1131589737955942405,britney_201904,2022-02-28 09:34:44.634058-05:00,2022-02-28 09:34:44.634058-05:00,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1131594909469892610,britney_201904,2022-02-28 09:34:44.627125-05:00,2022-02-28 09:34:44.627125-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1131594812694511617,britney_201904,2022-02-28 09:34:44.627227-05:00,2022-02-28 09:34:44.627227-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Initialize Elasticsearch client with host specification
es = Elasticsearch(hosts=["http://localhost:9200"])

# Ingest data into Elasticsearch
def ingest_data(df):
    for index, row in df.iterrows():
        es.index(index='tweets', body=row.to_dict())


In [14]:
def count_tweets_by_day(term):
    query = {
        "query": {
            "match": {
                "text": term
            }
        },
        "aggs": {
            "tweets_per_day": {
                "date_histogram": {
                    "field": "created_at",
                    "interval": "day"
                }
            }
        }
    }
    response = es.search(index='tweets', body=query)
    return response['aggregations']['tweets_per_day']['buckets']

def count_unique_users(term):
    query = {
        "query": {
            "match": {
                "text": term
            }
        },
        "aggs": {
            "unique_users": {
                "cardinality": {
                    "field": "user_id"
                }
            }
        }
    }
    response = es.search(index='tweets', body=query)
    return response['aggregations']['unique_users']['value']

def average_likes(term):
    query = {
        "query": {
            "match": {
                "text": term
            }
        },
        "aggs": {
            "average_likes": {
                "avg": {
                    "field": "likes"
                }
            }
        }
    }
    response = es.search(index='tweets', body=query)
    return response['aggregations']['average_likes']['value']

def tweets_by_place(term):
    query = {
        "query": {
            "match": {
                "text": term
            }
        },
        "aggs": {
            "places": {
                "terms": {
                    "field": "place_id"
                }
            }
        }
    }
    response = es.search(index='tweets', body=query)
    return response['aggregations']['places']['buckets']

def tweets_by_time_of_day(term):
    query = {
        "query": {
            "match": {
                "text": term
            }
        },
        "aggs": {
            "time_of_day": {
                "date_histogram": {
                    "field": "created_at",
                    "interval": "hour"
                }
            }
        }
    }
    response = es.search(index='tweets', body=query)
    return response['aggregations']['time_of_day']['buckets']

def top_user(term):
    query = {
        "query": {
            "match": {
                "text": term
            }
        },
        "aggs": {
            "top_users": {
                "terms": {
                    "field": "user_id",
                    "size": 1
                }
            }
        }
    }
    response = es.search(index='tweets', body=query)
    return response['aggregations']['top_users']['buckets'][0] if response['aggregations']['top_users']['buckets'] else None


In [17]:
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts=["http://localhost:9200"])

if not es.ping():
    print("Connection failed")
else:
    print("Connection successful")


Connection failed


In [25]:
import sys
print(sys.executable)


C:\ProgramData\anaconda3\python.exe


In [23]:
import sys
print(sys.executable)


C:\ProgramData\anaconda3\python.exe


In [28]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Function to count tweets by day
def count_tweets_by_day(term):
    query = {
        "query": {
            "match": {
                "text": term
            }
        },
        "aggs": {
            "tweets_per_day": {
                "date_histogram": {
                    "field": "created_at",
                    "calendar_interval": "day"
                }
            }
        }
    }
    response = es.search(index='tweets', body=query)
    return response['aggregations']['tweets_per_day']['buckets']

# Example search term
search_term = "music"

# Execute the query
try:
    print("Tweets per day:", count_tweets_by_day(search_term))
except Exception as e:
    print(f"Query error: {e}")


Query error: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x000001FDDB14B990>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it))


In [27]:
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts=["http://localhost:9200"])

try:
    if es.ping():
        print("Connection successful")
    else:
        print("Connection failed")
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")


Connection failed
